# Dubbing Assistant

1. Extract the audio from the video.
2. Separate the audio into vocals and accompaniment.
3. Transcribe the vocal. (Optional)
4. Translate the transcription. (Optional)
5. Text to speech
5. Clone the voice. (Future)
6. Use the cloned voice to read the translated script. (Future)

# Subtitle (YouTube)

If subtitles are available, we can skip the transcription and translation.

In [ ]:
%%capture
!pip install anvil-uplink

In [ ]:
import anvil.server

anvil.server.connect("server_2LPJFY3R2XW5CBQFJ2YEAVJI-MBGHABK6RX4NM5UT")

Connecting to wss://anvil.works/uplink
Anvil websocket open
Connected to "Default Environment" as SERVER


In [ ]:
%%capture
!pip install youtube-transcript-api

In [ ]:
%%capture
!pip install pytube

In [ ]:
from pytube import YouTube

In [ ]:
@anvil.server.callable
def get_video_title_from_url(url):
  try:
    yt = YouTube(url)
    return yt.title
  except:
    return None

In [ ]:
from youtube_transcript_api import YouTubeTranscriptApi

In [ ]:
# Get YouTube video id from its URL
def get_video_id_from_url(url):
  if "youtube.com/watch?v=" in url:
    video_id = url.split("v=")[1]
    if ("&" in video_id):
      video_id = video_id.split("&")[0]
    return video_id
  elif ("https://youtu.be/" in url):
    video_id = url.split("https://youtu.be/")[1]
    video_id = video_id.split("?")[0]
    return video_id
  elif ("youtube.com/shorts/" in url):
    video_id = url.split('youtube.com/shorts/')[1]
    if ('?' in video_id):
      video_id = video_id.split('?')[0]
    return video_id
  else:
    # Handle invalid URL
    print("Invalid URL!")
    return None

In [ ]:
def YouTube_transcript(video_id):
  try:
    transcript_list = YouTubeTranscriptApi.list_transcripts(video_id)
  except:
    transcript_list = []

  return transcript_list

In [ ]:
# print(YouTube_transcript('2bdvoVr64HI'))

In [ ]:
def video_Language_Code(numberOfTranscript, transcript_list):
  videoLanguageCode = "" # language-code of the default language of the video
  # translatableList = []
  if (numberOfTranscript > 0):
    for transcript in transcript_list: # iterate over all available transcripts
      if transcript.is_translatable and ("(auto-generated)" in transcript.language):
          videoLanguageCode = transcript.language_code
          break
    if videoLanguageCode == "":
      for transcript in transcript_list:
        if transcript.is_translatable:
          videoLanguageCode = transcript.language_code
          # translatableList.append((transcript.language, transcript.language_code))
          break
  return videoLanguageCode

In [ ]:
def video_Language_Code2(numberOfTranscript, transcript_list):
  videoLanguageCode = "" # language-code of the default language of the video
  translatableList = []
  if (numberOfTranscript > 0):
    for transcript in transcript_list: # iterate over all available transcripts
      if transcript.is_translatable and ("(auto-generated)" in transcript.language):
          videoLanguageCode = transcript.language_code
          break
    if videoLanguageCode == "":
      for transcript in transcript_list:
        if transcript.is_translatable:
          translatableList.append((transcript.language, transcript.language_code))
          # break
  if videoLanguageCode == '':
    if len(translatableList) == 1:
      videoLanguageCode = translatableList[0][1]
      return videoLanguageCode
    else:
      return translatableList
  else:
    return videoLanguageCode

In [ ]:
def fetchTranscript(code, transcript_list):
  selectedTranscript = ""
  selectedTranscript = transcript_list.find_transcript([code]).fetch()

  return selectedTranscript

In [ ]:
def fetchTranslation(videoLanguageCode, targetLanguageCode, transcript_list):
  selectedTranslation = ""
  selectedTranscript = transcript_list.find_transcript([videoLanguageCode])
  selectedTranslation = selectedTranscript.translate(targetLanguageCode).fetch()

  return selectedTranslation

In [ ]:
# def get_transcript2(url, code):
#   video_id = get_video_id_from_url(url)
#   transcript_list = YouTube_transcript(video_id)
#   # print(transcript_list)
#   selectedTranscript = []
#   if code == 'original':
#     try:
#       numberOfTranscript = len(list(enumerate(transcript_list)))
#       videoLanguageCode = video_Language_Code(numberOfTranscript, transcript_list)
#     except:
#       return None
#     return videoLanguageCode

In [ ]:
@anvil.server.callable
def get_transcript(url, code):
  video_id = get_video_id_from_url(url)
  transcript_list = YouTube_transcript(video_id)
  # print(transcript_list)
  selectedTranscript = []
  if code == 'original':
    try:
      numberOfTranscript = len(list(enumerate(transcript_list)))
      videoLanguageCode = video_Language_Code2(numberOfTranscript, transcript_list)
      if numberOfTranscript == 0:
        return []
      if isinstance(videoLanguageCode, list):
        return {'multiple': 'yes', 'options': videoLanguageCode}
      else:
        selectedTranscript = fetchTranscript(videoLanguageCode, transcript_list)
    except:
      return []

  elif code == 'zh-Hans':
    try:
      selectedTranscript = fetchTranscript(code, transcript_list)
    except:
      try:
        selectedTranscript = fetchTranscript('zh-Hant', transcript_list)
      except:
        try:
          selectedTranscript = fetchTranscript('zh-CN', transcript_list)
        except:
          try:
            selectedTranscript = fetchTranscript('zh-TW', transcript_list)
          except:
            try:
              numberOfTranscript = len(list(enumerate(transcript_list)))
              videoLanguageCode = video_Language_Code(numberOfTranscript, transcript_list)
              selectedTranscript = fetchTranslation(videoLanguageCode, code, transcript_list)
            except:
              return []
  else:
    try:
      selectedTranscript = fetchTranscript(code, transcript_list)
    except:
      try:
        numberOfTranscript = len(list(enumerate(transcript_list)))
        videoLanguageCode = video_Language_Code(numberOfTranscript, transcript_list)
        selectedTranscript = fetchTranslation(videoLanguageCode, code, transcript_list)
      except:
        return []
  for line in selectedTranscript:
    line['speaker'] = ''
    line['end'] = line['start'] + line['duration']
    line['start'] = round(line['start'])
    line['end'] = round(line['end'])
    line['startMin'] = line['start']//60
    line['startSec'] = line['start']%60
    line['endMin'] = line['end']//60
    line['endSec'] = line['end']%60
  keys_to_remove = ['start', 'end', 'duration']
  for line in selectedTranscript:
    for key in keys_to_remove:
      if key in line:
        del line[key]
  return selectedTranscript

In [ ]:
# print(get_transcript('https://youtu.be/2bdvoVr64HI?si=d6IUsv4vJCnw_Qxm', 'original'))

In [ ]:
# print(get_transcript('https://www.youtube.com/watch?v=XG7NfTwuGGs', 'de'))

# Download and split the YT audio

In [ ]:
# %%capture
# !pip install pytube

In [ ]:
%%capture
!pip install pydub

In [ ]:
from pytube import YouTube
from pydub import AudioSegment
import subprocess
import os
import requests
import base64

In [ ]:
def audioSplitPoints(duration_ms):
  split_points = [0]  # Split points in milliseconds

  if (duration_ms > 120*1000):
    duration_left = duration_ms
    point = 0
    while (duration_left > 120*1000):
      point = point + 120000
      duration_left = duration_left - 120000
      split_points.append(point)

    if (duration_left <= 60000):
      split_points.pop()

  return split_points

In [ ]:
def splitAudio(split_points, audio, duration_ms):
  audio_segments = []
  wav_files = []

  if (len(split_points) > 1):
    for i in range(len(split_points) - 1):
      start_time = split_points[i]
      end_time = split_points[i + 1]
      segment = audio[start_time:end_time]
      audio_segments.append(segment)

    segment = audio[split_points[-1]:duration_ms]
    audio_segments.append(segment)

    os.chdir("/content/audio")
    for i, segment in enumerate(audio_segments):
      output_file = f'segment_{i + 1}.wav'
      segment.export(output_file, format='wav')
      # print(f"Segment {i + 1} saved as {output_file}")
      wav_files.append(output_file)

    os.system("rm audio.wav")

  elif (len(split_points) == 1):
    os.chdir("/content/audio")
    os.rename('audio.wav', 'segment_1.wav')
    wav_files.append("segment_1.wav")

  return wav_files

In [ ]:
@anvil.server.callable
def YTdownloadSplitAudio(url):
  print("Starting...")
  yt = YouTube(url)
  audio_resolution_list = []
  for stream in yt.streams.filter(adaptive=True, mime_type="audio/mp4"):
    audio_resolution_list.append(int(stream.abr[:-4]))
  audio_resolution = str(max(audio_resolution_list)) + "kbps"
  Itag_audio = ""
  total_size_audio = ""
  for stream in yt.streams.filter(adaptive=True, mime_type="audio/mp4", abr=audio_resolution):
    Itag_audio = stream.itag
  stream_audio = yt.streams.get_by_itag(Itag_audio)
  directory_name = "audio"
  if not os.path.exists(directory_name):
    os.system(f"mkdir {directory_name}")
  print("Downloading audio...")
  stream_audio.download(output_path="/content/audio", filename="audio.wav")
  print("Downloaded")
  audio_file = '/content/audio/audio.wav'
  audio = AudioSegment.from_file(audio_file)
  duration_ms = len(audio)
  split_points = audioSplitPoints(duration_ms)
  print("Splitting...")
  wav_files = splitAudio(split_points, audio, duration_ms)
  return wav_files

In [ ]:
%%capture
!pip install demucs

In [ ]:
# Separate the audio into vocals and accompaniment
@anvil.server.callable
def vocalSeparation(url):
  wav_files = YTdownloadSplitAudio(url)
  print("Splitting stage completed")
  demucs_command = "demucs --two-stems vocals -n htdemucs_ft"
  demucs_counter = 0
  input_vocals = []
  input_non_vocals = []

  os.chdir("/content/audio")
  print("Separating vocal and non-vocal...")
  for wav_file in wav_files:
      # Construct the full command
      full_command = f"{demucs_command} {wav_file}"

      # Execute the command
      try:
          subprocess.run(full_command, shell=True, check=True)
          # print(f"Processing {wav_file} completed successfully.")
          input_vocals.append(f'/content/audio/separated/htdemucs_ft/{wav_file[:-4]}/vocals.wav')
          input_non_vocals.append(f'/content/audio/separated/htdemucs_ft/{wav_file[:-4]}/no_vocals.wav')
          demucs_counter = demucs_counter + 1
      except subprocess.CalledProcessError as e:
          print(f"Error processing {wav_file}: {e}")

  if (demucs_counter == len(wav_files)):
    os.chdir("/content")
    return [input_vocals, input_non_vocals]

  else:
    os.chdir("/content")
    return "Separation is unsuccessful"

In [ ]:
# from google.colab import drive
# drive.mount('/content/drive')

In [ ]:
# Combining the splitted segments back into a long audio file
@anvil.server.callable
def combineAudio(url):
  input = vocalSeparation(url)
  print("Separation stage completed")
  print("Recombining splitted segments...")
  input_vocals = input[0]
  input_non_vocals = input[1]

  directory_name = "combinedAudio"
  if not os.path.exists(directory_name):
    os.system(f"mkdir {directory_name}")

  audio_track = []
  for i in input_vocals:
    audio_track.append(AudioSegment.from_file(i))

  vocal = audio_track[0]
  for j in range(len(audio_track)):
    if (j > 0):
      vocal = vocal + audio_track[j]
  vocal.export("/content/combinedAudio/vocal.wav", format="wav")
  vocal = AudioSegment.from_wav("/content/combinedAudio/vocal.wav")
  vocal.export("/content/combinedAudio/vocal.mp3", format="mp3")

  audio_track = []
  for i in input_non_vocals:
    audio_track.append(AudioSegment.from_file(i))

  nonVocal = audio_track[0]
  for j in range(len(audio_track)):
    if (j > 0):
      nonVocal = nonVocal + audio_track[j]
  nonVocal.export("/content/combinedAudio/nonVocal.wav", format="wav")
  nonVocal = AudioSegment.from_wav("/content/combinedAudio/nonVocal.wav")
  nonVocal.export("/content/combinedAudio/nonVocal.mp3", format="mp3")
  os.system("rm -r /content/audio")

In [ ]:
# !rm -r /content/audio

In [ ]:
# combineAudio("https://www.youtube.com/watch?v=s6jDsPXmo2I")

# Upload accompaniment

In [ ]:
from anvil.tables import app_tables

In [ ]:
@anvil.server.callable
def upload_accompaniment_vocal(videoUrl, videoTitle):
  videoID = get_video_id_from_url(videoUrl)
  videoUrl = 'https://www.youtube.com/watch?v=' + videoID
  videoRow = app_tables.videos.get(youTubeVideoID=videoID)
  if videoRow is None:
    app_tables.videos.add_row(videoTitle=videoTitle, youTubeVideoID=videoID, videoUrl=videoUrl)
  videoRow = app_tables.videos.get(youTubeVideoID=videoID)
  if videoRow['accompaniment'] is None:
    with open("/content/combinedAudio/nonVocal.mp3", "rb") as f:
      audioBytes = f.read()
    audio = anvil.BlobMedia(content_type="audio/mp3", content=audioBytes, name='accompaniment.mp3')
    videoRow.update(accompaniment=audio)
  if videoRow['vocal'] is None:
    with open("/content/combinedAudio/vocal.mp3", "rb") as f:
      audioBytes = f.read()
    audio = anvil.BlobMedia(content_type="audio/mp3", content=audioBytes, name='vocal.mp3')
    videoRow.update(vocal=audio)
  os.system("rm -r /content/combinedAudio")

In [ ]:
def upload_accompaniment_vocal2(videoUrl, videoTitle):
  videoID = get_video_id_from_url(videoUrl)
  videoUrl = 'https://www.youtube.com/watch?v=' + videoID
  videoRow = app_tables.videos.get(youTubeVideoID=videoID)
  if videoRow is None:
    app_tables.videos.add_row(videoTitle=videoTitle, youTubeVideoID=videoID, videoUrl=videoUrl)
  videoRow = app_tables.videos.get(youTubeVideoID=videoID)
  if videoRow['accompaniment'] is None:
    with open("/content/combinedAudio/nonVocal.mp3", "rb") as f:
      audioBytes = f.read()
    audio = anvil.BlobMedia(content_type="audio/mp3", content=audioBytes, name='accompaniment.mp3')
    videoRow.update(accompaniment=audio)
  if videoRow['vocal'] is None:
    with open("/content/combinedAudio/vocal.mp3", "rb") as f:
      audioBytes = f.read()
    audio = anvil.BlobMedia(content_type="audio/mp3", content=audioBytes, name='vocal.mp3')
    videoRow.update(vocal=audio)
  # os.system("rm -r /content/combinedAudio")

# Transcribe the vocal using Whisper

In [ ]:
# !mkdir /content/combineAudio

In [ ]:
%%capture
!pip install git+https://github.com/openai/whisper.git
!sudo apt update && sudo apt install ffmpeg

In [ ]:
import whisper
import json

In [ ]:
# import json

In [ ]:
import anvil.media

In [ ]:
# !whisper --model large-v2 --output_dir /content --output_format json --task translate /content/combineAudio/vocal.mp3

In [ ]:
# !whisper --help

In [ ]:
@anvil.server.callable
def transcribe(url, videoTitle):
  if os.path.exists('/content/combinedAudio'):
    os.system('rm -r /content/combinedAudio')

  videoID = get_video_id_from_url(url)
  videoRow = app_tables.videos.get(youTubeVideoID=videoID)
  if videoRow is not None:
    if videoRow['vocal'] is not None:
      os.system(f"mkdir /content/combinedAudio")
      vocal = AudioSegment.from_file(io.BytesIO(videoRow['vocal'].get_bytes()), format="mp3")
      vocal.export("/content/combinedAudio/vocal.wav", format="wav")
    else:
      combineAudio(url)
  else:
    combineAudio(url)

  upload_accompaniment_vocal2(url, videoTitle)
  print("Transcribing...")
  if os.path.exists("/content/vocal.json"):
    os.remove("/content/vocal.json")

  transcript_command = 'whisper --model large-v2 --output_dir /content --output_format json --task transcribe /content/combinedAudio/vocal.wav'
  subprocess.run(transcript_command, shell=True, check=True)

  with open('vocal.json', 'r') as file:
    # Load the JSON data into a Python dictionary
    transcript = json.load(file)

  transcriptGenerated = []
  for segment in transcript['segments']:
    transcriptGenerated.append({"start": segment['start'], "end": segment['end'], "text": segment['text']})

  print("\nTranscription stage completed")
  for line in transcriptGenerated:
    line['speaker'] = ''
    line['start'] = round(line['start'])
    line['end'] = round(line['end'])
    line['startMin'] = line['start']//60
    line['startSec'] = line['start']%60
    line['endMin'] = line['end']//60
    line['endSec'] = line['end']%60
  keys_to_remove = ['start', 'end']
  for line in transcriptGenerated:
    for key in keys_to_remove:
      if key in line:
        del line[key]
  return transcriptGenerated

In [ ]:
# transcribe('https://www.youtube.com/shorts/ikjrjIwGcNY', '金門的年輕人在大選前如何掙扎？－ BBC News 中文')

# Conversion between transcript list and Anvil media and SRT/TXT file

In [ ]:
@anvil.server.callable
def convert_transcript_to_anvil_media(transcript):
  srt_content = ""
  counter = 1

  for line in transcript:
    line['startMin'] = int(line['startMin'])
    line['startSec'] = int(line['startSec'])
    line['endMin'] = int(line['endMin'])
    line['endSec'] = int(line['endSec'])
    start_time = "{:02d}:{:02d}:{:02d},{}".format(0, line['startMin'], line['startSec'], 0)
    end_time = "{:02d}:{:02d}:{:02d},{}".format(0, line['endMin'], line['endSec'], 0)

    srt_content += str(counter) + "\n"
    srt_content += start_time + " --> " + end_time + "\n"
    srt_content += line['text'] + "\n\n"
    counter += 1

  with open('transcript.srt', 'w') as file:
    file.write(srt_content)

  with open('transcript.srt', 'rb') as f:
    transcriptByte = f.read()
  transcriptMedia = anvil.BlobMedia(content_type="text/plain", content=transcriptByte, name="transcript.srt")
  return transcriptMedia

In [ ]:
@anvil.server.callable
def convert_transcript_to_txt(transcript):
  srt_content = ""
  counter = 1

  for line in transcript:
    start_time = "{:02d}:{:02d}:{:02d},{}".format(0, line['startMin'], line['startSec'], 0)
    end_time = "{:02d}:{:02d}:{:02d},{}".format(0, line['endMin'], line['endSec'], 0)

    srt_content += str(counter) + "\n"
    srt_content += start_time + " --> " + end_time + "\n"
    srt_content += line['text'] + "\n\n"
    counter += 1

  with open('transcript.txt', 'w') as file:
    file.write(srt_content)

In [ ]:
@anvil.server.callable
def convert_srt_to_list(srt_file):
  transcript_list = []
  with open(srt_file, 'r') as file:
    lines = file.readlines()
    i = 0
    while i < len(lines):
      line = lines[i].strip()
      if line.isdigit():
        i += 1  # Skip the number line
        time_line = lines[i].strip().split(" --> ")
        start_time = time_line[0].split(':')
        end_time = time_line[1].split(':')
        start_min = int(start_time[1])
        end_min = int(end_time[1])
        start_sec, start_mic_sec = map(int, start_time[2].split(',')[0:2])
        end_sec, end_mic_sec = map(int, end_time[2].split(',')[0:2])
        start_sec = round(start_sec + start_mic_sec/1000)
        end_sec = round(end_sec + end_mic_sec/1000)
        text = lines[i + 1].strip()
        transcript_list.append({
            'speaker': '',
            'startMin': start_min,
            'startSec': start_sec,
            'endMin': end_min,
            'endSec': end_sec,
            'text': text
        })
        i += 2
      else:
        i += 1
  return transcript_list

In [ ]:
# Example usage:
# srt_file = "transcript.srt"
# transcript_list = convert_srt_to_list(srt_file)
# for line in transcript_list:
#   print(line)


# Translate

In [ ]:
%%capture
!pip install deep-translator

In [ ]:
from deep_translator import GoogleTranslator

In [ ]:
@anvil.server.callable
def translate_txt(transcript, targetLanguageCode):
  convert_transcript_to_txt(transcript)

  if targetLanguageCode == 'zh-Hans':
    targetLanguageCode = 'zh-CN'

  translate_srt = GoogleTranslator(source='auto', target=targetLanguageCode).translate_file('transcript.txt')
  with open('translation.srt', 'w') as file:
    file.write(translate_srt)

  if os.path.exists("/content/vocal.json"):
    os.remove("/content/vocal.json")
  if os.path.exists("/content/transcript.txt"):
    os.remove("/content/transcript.txt")

  translation = convert_srt_to_list('translation.srt')
  if os.path.exists("/content/transcript.srt"):
    os.remove("/content/transcript.srt")

  os.remove('/content/translation.srt')

  return translation

In [ ]:
# print(result)

#Text to Speech

In [ ]:
%%capture
!pip install edge-tts

In [ ]:
import subprocess
import io

In [ ]:
def execute_command(command):
  try:
    output = subprocess.check_output(command, shell=True, universal_newlines=True)
    # Split the output into a list of strings
    output_list = output.splitlines()
    return output_list
  except subprocess.CalledProcessError as e:
    print("Error executing command:", e)
    return None

In [ ]:
command = "edge-tts --list-voices"

# Execute the command and store the output in a list
voice_list = execute_command(command)

# Remove empty strings using list comprehension
filtered_list = [x for x in voice_list if x != ""]

# Initialize an empty list to store dictionaries
result = []

# Iterate over the list of strings
for i in range(0, len(filtered_list), 2):
    # Extract name and gender
    name = filtered_list[i].split(': ')[1]
    gender = filtered_list[i + 1].split(': ')[1][0].lower()  # Extract first character and convert to lowercase

    # Create dictionary and append to result list
    result.append({'name': name, 'gender': gender})

voice_list = []

for item in result:
    name_parts = item['name'].split('-')
    language = name_parts[0]
    country = name_parts[1]
    item_with_language_country = {
        'language': language,
        'country': country.lower(),
        'name': item['name'],
        'gender': item['gender']
    }
    voice_list.append(item_with_language_country)

print(voice_list)

[{'language': 'af', 'country': 'za', 'name': 'af-ZA-AdriNeural', 'gender': 'f'}, {'language': 'af', 'country': 'za', 'name': 'af-ZA-WillemNeural', 'gender': 'm'}, {'language': 'am', 'country': 'et', 'name': 'am-ET-AmehaNeural', 'gender': 'm'}, {'language': 'am', 'country': 'et', 'name': 'am-ET-MekdesNeural', 'gender': 'f'}, {'language': 'ar', 'country': 'ae', 'name': 'ar-AE-FatimaNeural', 'gender': 'f'}, {'language': 'ar', 'country': 'ae', 'name': 'ar-AE-HamdanNeural', 'gender': 'm'}, {'language': 'ar', 'country': 'bh', 'name': 'ar-BH-AliNeural', 'gender': 'm'}, {'language': 'ar', 'country': 'bh', 'name': 'ar-BH-LailaNeural', 'gender': 'f'}, {'language': 'ar', 'country': 'dz', 'name': 'ar-DZ-AminaNeural', 'gender': 'f'}, {'language': 'ar', 'country': 'dz', 'name': 'ar-DZ-IsmaelNeural', 'gender': 'm'}, {'language': 'ar', 'country': 'eg', 'name': 'ar-EG-SalmaNeural', 'gender': 'f'}, {'language': 'ar', 'country': 'eg', 'name': 'ar-EG-ShakirNeural', 'gender': 'm'}, {'language': 'ar', 'coun

In [ ]:
def speak_preparation(output_list):
  '''Fill in the empty speakers in the transcript'''

  if output_list[0]['speaker'] == '':
    output_list[0]['speaker'] = 'm1'

  for i in range(len(output_list)):
    if output_list[i]['speaker'] == '':
      output_list[i]['speaker'] = output_list[i-1]['speaker']

  return output_list

In [ ]:
def speak_preparation2(languageCode, transcript):
  '''Make sure the number of speakers is not more than the number of available voices'''

  if languageCode == 'zh-Hans':
    languageCode = 'zh'

  # Extract unique 'speaker' values
  unique_speakers = set()
  for line in transcript:
    unique_speakers.add(line['speaker'])

  unique_speakers = list(unique_speakers)

  if len(unique_speakers) > 1:
    maleSpeakers = []
    femaleSpeakers = []
    for item in unique_speakers:
      if item.startswith('m'):
        maleSpeakers.append(item)
      elif item.startswith('f'):
        femaleSpeakers.append(item)

    # Count the number of voices with 'language' key equal to languageCode
    maleVoiceCount = sum(1 for voice in voice_list if voice.get('language') == languageCode and voice.get('gender') == 'm')
    femaleVoiceCount = sum(1 for voice in voice_list if voice.get('language') == languageCode and voice.get('gender') == 'f')

    if (maleVoiceCount == 0 and len(maleSpeakers) > 0) or (femaleVoiceCount == 0 and len(femaleSpeakers) > 0):
      return 'Voice not found'

    while maleVoiceCount < len(maleSpeakers):
      maleSpeakers.pop(-1)

    while femaleVoiceCount < len(femaleSpeakers):
      femaleSpeakers.pop(-1)

    for line in transcript:
      if line['speaker'][0] == 'm' and (line['speaker'] not in maleSpeakers):
        line['speaker'] = 'm1'
      if line['speaker'][0] == 'f' and (line['speaker'] not in femaleSpeakers):
        line['speaker'] = 'f1'

  return transcript

In [ ]:
def speak_preparation3(languageCode, transcript):
  '''Assign the voices to the lines in transcript'''

  if languageCode == 'zh-Hans':
    languageCode = 'zh'

  # selectedVoiceList = []

  speakers = {'m1': [], 'f1': [], 'm2': [], 'f2': [], 'm3': [], 'f3': [], 'm4': [], 'f4': [], 'm5': [], 'f5': [], 'm6': [], 'f6': []}

  for line in transcript:
    key = line.get('speaker')
    if key in speakers:
      speakers[key].append(line)

  speakerKeys = ['m1', 'f1', 'm2', 'f2', 'm3', 'f3', 'm4', 'f4', 'm5', 'f5', 'm6', 'f6']
  for key in speakerKeys:
    speakerNumber = int(key[1])
    if len(speakers[key]) > 0:
      for line in speakers[key]:
        for voice in voice_list:
          if voice['language'] == languageCode and voice['gender'] == line['speaker'][0]:
            if speakerNumber == 1:
              line['voice'] = voice['name']
              break
            else:
              speakerNumber -= 1

  # Flatten the dictionaries into a single list
  all_dicts = [item for sublist in speakers.values() for item in sublist]

  # Sort the list based on the value of 'start' key in each dictionary
  transcriptWithVoice = sorted(all_dicts, key=lambda x: x['start'])

  return transcriptWithVoice

In [ ]:
def speak_preparation4(transcriptWithVoice):
  '''Text-to-speech starts'''

  i = 0
  speeches = []
  for line in transcriptWithVoice:
    # Convert text to speech with estimated duration
    if line['text'] != '':
      text = line['text']
      voice = line['voice']
      rate = 0
      command = f'edge-tts --voice {voice} --text "{text}" --write-media {"/content/audio/tts" + str(i) + ".mp3"}'
      subprocess.run(command, shell=True, check=True)
      audio_file = f"/content/audio/tts{str(i)}.mp3"
      audio = AudioSegment.from_file(audio_file)
      duration = len(audio)/1000
      rate = 10
      while duration > line['duration']:
        os.remove(audio_file)
        command = f'edge-tts --rate=+{rate}% --voice {voice} --text "{text}" --write-media {"/content/audio/tts" + str(i) + ".mp3"}'
        subprocess.run(command, shell=True, check=True)
        audio_file = f"/content/audio/tts{str(i)}.mp3"
        audio = AudioSegment.from_file(audio_file)
        duration = len(audio)/1000
        rate += 10
        if rate > 200:
          break
    else:
      audio = AudioSegment.silent(duration=line['duration']*1000)
    speeches.append(audio)
    i += 1
  # return len(speech)

  if transcriptWithVoice[0]['start'] == 0:
    speech = speeches[0]
  else:
    speech = AudioSegment.silent(duration=transcriptWithVoice[0]['start']*1000)
    speech = speech + speeches[0]

  length = len(speech)
  silenceDuration = transcriptWithVoice[1]['start']*1000 - length
  if silenceDuration > 0:
    speech = speech + AudioSegment.silent(duration=silenceDuration)

  for j in range(1, len(speeches)-1):
    speech = speech + speeches[j]
    length = len(speech)
    silenceDuration = transcriptWithVoice[j+1]['start']*1000 - length
    if silenceDuration > 0:
      speech = speech + AudioSegment.silent(duration=silenceDuration)

  speech = speech + speeches[-1]
  speech.export("/content/speech.mp3", format="mp3")
  return speech

In [ ]:
@anvil.server.callable
def speak(languageCode, user, videoID):
  if os.path.exists('/content/transcript.srt'):
    os.remove('/content/transcript.srt')

  videoRow = app_tables.videos.get(youTubeVideoID=videoID)
  transcriptRow = app_tables.transcripts.get(createdBy=user, languageCode=languageCode, video=videoRow)
  transcript = transcriptRow['transcript']
  output_list = []

  for item in transcript:
    item['startMin'] = int(item['startMin'])
    item['startSec'] = int(item['startSec'])
    item['endMin'] = int(item['endMin'])
    item['endSec'] = int(item['endSec'])
    duration = (item['endMin'] * 60 + item['endSec']) - (item['startMin'] * 60 + item['startSec'])
    new_item = {
        "text": item["text"],
        "speaker": item["speaker"],
        "start": item['startMin'] * 60 + item['startSec'],
        "end": item['endMin'] * 60 + item['endSec'],
        "duration": duration
    }
    output_list.append(new_item)

  if output_list[0]['start'] < 0:
    output_list[0]['start'] = 0

  transcript = speak_preparation(output_list)
  transcript = speak_preparation2(languageCode, transcript)
  if transcript == 'Voice not found':
    return 'Voice not found'

  directory_name = "audio"
  if not os.path.exists(directory_name):
    os.system(f"mkdir {directory_name}")
  else:
    os.system(f"rm -r {directory_name}")
    os.system(f"mkdir {directory_name}")

  if os.path.exists('/content/speech.mp3'):
    os.remove('/content/speech.mp3')

  transcriptWithVoice = speak_preparation3(languageCode, transcript)

  speech = speak_preparation4(transcriptWithVoice)

  accompaniment = AudioSegment.from_file(io.BytesIO(videoRow['accompaniment'].get_bytes()), format="mp3")

  # Ensure both audio tracks start at the same time
  speech = speech.set_frame_rate(accompaniment.frame_rate)
  speech = speech.set_channels(accompaniment.channels)
  speech = speech.set_sample_width(accompaniment.sample_width)

  # Overlay the audio onto the background
  dub = accompaniment.overlay(speech)
  # Export the resulting audio
  dub.export("dub.mp3", format="mp3")
  with open("/content/dub.mp3", "rb") as f:
    audioBytes = f.read()
  audio = anvil.BlobMedia(content_type="audio/mp3", content=audioBytes, name='dub.mp3')
  transcriptRow.update(dub=audio, accent='Machine')
  os.system("rm -r audio")
  os.remove('/content/speech.mp3')
  os.remove('/content/dub.mp3')

  return True

In [ ]:
# user = app_tables.users.get(userID='u2')
# speak('en', user, '2bdvoVr64HI')

#Sandbox

## Speaker Diarization

## Clone the voice